# Módulo 1: Análisis de datos en el ecosistema Python

### Sesión (26)

# Machine Learning Time Series Forecasting

Para aplicar modelos de aprendizaje automático (___Machine Learning___) a problemas de predicción de series temporales, **debemos transformar la serie en una matriz** en la que cada valor esté asociado a la ventana temporal con las observaciones anteriores (lags) correspondientes. Para ello tenemos que crear conjuntos de datos con las observaciones pasadas acompañadas con el último valor de la serie después de la ventana temporal.

![transform_timeseries.gif](attachment:transform_timeseries.gif)

De este modo, conseguimos **convertir el problema de forecasting** o la predicción de los valores de una serie temporal **a un problema tipo regresión**, donde el modelo entrenado es capaz de calcular el próximo valor de una serie **teniendo las secuencias anteriores según el tamaño de la ventana temporal elegida** en la fase preparación de datos.  

Una vez que los datos se han reorganizado en la nueva forma, se puede **entrenar cualquier modelo de regresión** para predecir el próximo valor de la serie. Durante el entrenamiento del modelo, cada fila se considera una instancia o conjunto de datos, donde los valores en los retrasos $1, 2, ... p$ se consideran como **variables independientes o predictores** de la cantidad de la serie temporal en el paso de tiempo $p+1$.

![diagram-trainig-forecaster.png](attachment:diagram-trainig-forecaster.png)

### Recursive multi-step forecasting


Dado que se requiere el valor $t_{(n-1)}$ para predecir $t_{(n)}$, cuando se desconoce $t_{(n-1)}$, podemos **aplicar un proceso recursivo** en el que, cada nuavo valor calculado, se basa en la anterior. Este proceso se conoce como predicción recursiva (___recursive forecasting___).

![diagram-recursive-mutistep-forecasting.png](attachment:diagram-recursive-mutistep-forecasting.png)

### Direct multi-step forecasting


La **predicción directa** de varios pasos consiste en **entrenar un modelo diferente para cada paso del horizonte** de predicción. Como resultado, las predicciones son independientes entre sí.

![diagram-direct-multi-step-forecasting.png](attachment:diagram-direct-multi-step-forecasting.png)

In [ ]:
# importamos las librerías necesarias
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
# Modificamos los parámetros de los gráficos en matplotlib
from matplotlib.pyplot import rcParams

rcParams['figure.figsize'] = 12, 6 # el primer dígito es el ancho y el segundo el alto
rcParams["font.weight"] = "bold"
rcParams["font.size"] = 10
rcParams["axes.labelweight"] = "bold"

### Airline Passenger Dataset

Importamos los datos del ejemplo disponible en la librería ___seaborn___ que contiene el número total de pasajeros aéreos de forma mensual.

In [ ]:
import seaborn as sns
import pandas as pd

# Cargar el dataset de "flights"
df_air = sns.load_dataset('flights')
df_air

In [ ]:
# Consultamos la información del dataset descargado
df_air.info()

In [ ]:
# Crear una nueva columna con la unificación de otras dos
df_air['year_month'] = df_air.apply(lambda x: str(x['year']) + '-' + x['month'], axis=1)

# Convertir la columna en fechas tipo DatetimeIndex
df_air['fechas'] = pd.to_datetime(df_air['year_month'], format='%Y-%b')

# Convertir la columna de fechas a los índices del DataFrame e indicar que los datos son "mensuales"
df_air.set_index('fechas', inplace=True)
df_air.index.freq = 'MS'

# Quitar las columnas no necesarias
df_air.drop(columns=['year', 'month', 'year_month'], inplace=True)

df_air

In [ ]:
# Visualizar el DataFrame creado con los datos de la serie temporal
plt.plot(df_air)
plt.show()

In [ ]:
# Definir el periodo de prueba (horizonte de predicción)
horizonte = 12  # La cantidad de puntos a predecir
df_test = df_air.tail(horizonte)
df_test

In [ ]:
# Filtrar la serie original para sacar el periodo de entrenamiento
df_train = df_air[df_air.index.isin(df_test.index)==False]
df_train

### Recursive autoregressive forecasting

__`Skforecast`__ es una libraría de _Python_ para la **previsión de series temporales basada en _scikit-learn_**. Este paquete proporciona un conjunto de herramientas y modelos para trabajar con datos de series temporales, incluido el preprocesamiento, la ingeniería de variables y la predicción automatizada.

La principal ventaja de _skforecast_ es que permite el uso de la conocida interfaz _scikit-learn_ para **convertir los problemas de previsión de series temporales en problemas de regresión** y de este modo poder **aplicar distintas técnicas de Machine Learning**.

In [ ]:
pip install skforecast

Creamos un modelo autoregresivo usando bosques aleatrorios (_random forest_)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg


mod_rf = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=77),
                lags = 12
                )

mod_rf.fit(y=df_train['passengers'])
mod_rf

In [ ]:
pred_rf = mod_rf.predict(steps=horizonte)
pred_rf

In [ ]:
pred_rf = pred_rf.round()
pred_rf

In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_rf, label='Predicción - RF')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_rf))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_rf)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_rf))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_rf)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_rf))

##### **In-Sample** Model Evaluation

In [ ]:
# Crear los vectores de entrenamiento
X_train, y_train = mod_rf.create_train_X_y(df_train['passengers'])

# Calcular los valores del modelo en el periodo de entrenamiento
fitted_values = mod_rf.regressor.predict(X_train)
fitted_values

In [ ]:
# Ordenar las predicciones del modelo para el periodo de emtrenamiento
estim_rf = pd.Series(data=np.zeros(df_train.size), index=df_train.index, name='fitted')
estim_rf[:horizonte] = np.nan
estim_rf[horizonte:] = fitted_values
estim_rf

In [ ]:
plt.plot(df_train, label='Entrenamiento')
plt.plot(estim_rf, label='Estimación RandomForestRegressor')
plt.title("Datos reales vs. Estimación del modelo (In-Sample forecasting)")
plt.legend()
plt.show()

In [ ]:
# Comparar los valores reales con la estimación del modelo
sns.scatterplot(x=df_train['passengers'], y=estim_rf)
plt.plot(estim_rf, estim_rf, color='r', linestyle=':')
plt.title("Valores reales vs. valors estimados (In-Sample forecasting) RandomForestRegressor")
plt.show()

In [ ]:
# Calcular los valores del componente residual (In-sample errors)
resid_rf = df_train['passengers']-estim_rf
resid_rf

In [ ]:
# Las estadísticas del componente residual
resid_rf.describe().round(3)

In [ ]:
plt.plot(resid_rf)
plt.title("Componente residual del modelo RandomForestRegressor (In-Sample fitted errores)")
plt.axhline(y=0, color='r', linestyle=':')
plt.show()

In [ ]:
# El histograma del componente residual (In-sample errors)
sns.histplot(data=resid_rf, bins=50)
plt.show()

In [ ]:
win = 20
resid_rf_std = resid_rf.rolling(win).std().iloc[win-1::win]
plt.plot(resid_rf_std, label='Desviación estándar')
plt.axhline(y=resid_rf.std(), color='r', linestyle='--')
plt.title("Características estadísticas: Residual - RandomForestRegressor")
plt.ylim(0,50)
plt.legend()
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(resid_rf.dropna(), lags=37)
plt.xticks(np.arange(37))
plt.ylim(-1.1,1.1)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

plot_pacf(resid_rf.dropna(), lags=37, method='ywm')
plt.xticks(np.arange(37))
plt.ylim(-1.1,1.1)
plt.show()

Vemos que hay algo de correlación presente en los valores del componente residual

In [ ]:
# Analizamos el componente residual

# Coeficiente de correlación entre valores reales y los errores
print(df_train['passengers'].corr(resid_rf).round(4))

# Coeficiente de correlación entre valores estimados y los errores
print(estim_rf.corr(resid_rf).round(4))

sns.scatterplot(x=df_train['passengers'], y=resid_rf)
plt.title("Valores reales versus valores residuales - RandomForestRegressor")
plt.show()

In [ ]:
sns.scatterplot(x=estim_rf, y=resid_rf)
plt.title("Valores estimados versus valores residuales - RandomForestRegressor")
plt.show()

#### Analizar el intervalo de confianza

In [ ]:
conf_rf = mod_rf.predict_interval(steps=horizonte, interval=[0,100], random_state=111)
conf_rf.index = df_test.index
conf_rf

Sacamos la banda que se aplica como intervalo de confianza: _delta_

In [ ]:
conf_rf['delta'] = conf_rf.apply(lambda x: x['upper_bound'] - x['lower_bound'], axis=1)
conf_rf

In [ ]:
plt.plot(conf_rf['delta'])
plt.title("Evolución del rango de los intervalos de confianza  - RandomForestRegressor")
plt.show()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_train.index,
    y=df_train['passengers'],
    name="Entrenamiento",
    mode="lines"
    ))

fig.add_trace(go.Scatter(
    x=df_test.index,
    y=df_test['passengers'],
    name="Test",
    mode="lines"
    ))


fig.add_trace(go.Scatter(
    x=pred_rf.index,
    y=pred_rf,
    name="Predicción (RandomForestRegressor - 12)",
    mode="markers+lines"
    ))

fig.add_trace(go.Scatter(
    x=conf_rf.index,
    y=conf_rf['lower_bound'],
    name="lower",
    mode="lines",
    line=dict(width=0),
    showlegend=False
    ))

fig.add_trace(go.Scatter(
    x=conf_rf.index,
    y=conf_rf['upper_bound'],
    name="upper",
    mode="lines",
    line=dict(width=0),
    showlegend=False,
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty'
    ))

fig.update_layout(title="Número de pasajeros aéreos de cada mes desde el año 1949 al 1960",
                  title_font_size=22,
                  xaxis_title = 'Fecha',
                  yaxis_title= 'Pasajeros'
                  )

fig.show()

### Ajustar los hiperparámetros


Una técnica común para encontrar la ventana de tiempo que marca los retrasos necesarios para modelar la serie temporal consiste en analizar el _correlograma_ normal y _parcial_

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(df_train, lags=20)
plt.xticks(np.arange(20))
plt.ylim(-1.1,1.1)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

plot_pacf(df_train, lags=50, method='ywm')
plt.xticks(np.arange(50))
plt.ylim(-1.1,1.1)
plt.show()

Podemos considerar una serie de posibles retrasos o _lags_ que sean importantes de cara al modelo:
- $1, 2, 3, ...., 25$
- $1, 2, 9, 12, 13$

In [ ]:
# Consideramos un rango para asignar el hiperparámetro
hiper_param = np.arange(1,26).tolist()
hiper_param.append([1,2,9,12,13])

# Generamos previamente los vectores necesarios para ir calculando y guardando el rendimiento
test_r2 = np.zeros(len(hiper_param))

for i in range(len(hiper_param)):
    # Generamos un modelo para cada hiperparámetro, lo entrenamos y calculamos el R_cuadrado sobre datos de test
    mod_bosque = ForecasterAutoreg(
                    regressor = RandomForestRegressor(random_state=77),
                    lags = hiper_param[i]
                )

    mod_bosque.fit(y=df_train['passengers'])
    test_r2[i] = r2_score(df_test, mod_bosque.predict(steps=horizonte).round())

print("El mejor valor de lags podría ser =", hiper_param[np.argmax(test_r2)],
      " que consigue un R2 =", max(test_r2))

# Graficamos el R_cuadrado de test
fig = plt.figure(figsize=(20,7))
plt.plot(list(map(str, hiper_param)), test_r2, linewidth=3, label='Test R^2')
plt.plot(str(hiper_param[np.argmax(test_r2)]), max(test_r2),
         marker='o', color = "red", label="max R^2")

plt.xlabel('Complejidad (lags)')
plt.ylabel('R2')
plt.legend(loc = 'lower right')
plt.show()



La librería _skforecast_ cuanta con un método propio llamado `grid_search_forecaster` que es una implementación de _GridSearch_ para encontrar la combinación óptima de los hiperparámetros.

In [ ]:
from skforecast.model_selection import grid_search_forecaster

# Modelo
forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=77),
                lags      = 12
             )

# Lags
lags_grid = [12, 13, [1,2,9,12,13]]

# Parámetros del regresor
param_grid = {'n_estimators': [100, 300, 500],
              'max_depth': [5, 10, 20, 30]}

results_grid = grid_search_forecaster(
                        forecaster         = forecaster,
                        y                  = df_train['passengers'],
                        param_grid         = param_grid,
                        lags_grid          = lags_grid,
                        steps              = horizonte,
                        refit              = False,
                        metric             = 'mean_absolute_error',
                        initial_train_size = int(len(df_train)*0.5),
                        fixed_train_size   = False,
                        return_best        = True,
                        verbose            = False
               )

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg


mod_rf_grid = ForecasterAutoreg(
                regressor = RandomForestRegressor(n_estimators=500,
                                                  max_depth=5,
                                                  random_state=77),
                lags = [1,2,9,12,13]
                )

mod_rf_grid.fit(y=df_train['passengers'])
pred_rf_grid = mod_rf_grid.predict(steps=horizonte).round()

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_rf_grid))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_rf_grid)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_rf_grid))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_rf_grid)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_rf_grid))


In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_rf_grid, label='Predicción - RF (grid)')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

Además de este método podemos usar el análisis de complejidad como problemas de regresión para ver la evolución al cambiar cada hiperparámetro.

In [ ]:
# Consideramos un rango para asignar el hiperparámetro
hiper_param = np.arange(5,501,5)

# Generamos previamente los vectores necesarios para ir calculando y guardando el rendimiento
test_r2 = np.zeros(hiper_param.size)

for i in range(hiper_param.size):
    # Generamos un modelo para cada hiperparámetro
    mod_bosque = ForecasterAutoreg(
                    regressor = RandomForestRegressor(n_estimators=hiper_param[i],
                                                      random_state=77),
                lags = [1,2,9,12,13]
                )

    mod_bosque.fit(y=df_train['passengers'])
    test_r2[i] = r2_score(df_test, mod_bosque.predict(steps=horizonte).round())

print("El mejor valor de n_estimator podría ser =", hiper_param[np.argmax(test_r2)],
      " que consigue un R2 =", max(test_r2))

# Graficamos el R_cuadrado
fig = plt.figure(figsize=(20,7))
plt.plot(hiper_param, test_r2, linewidth=3, label='Test R^2')
plt.plot(hiper_param[np.argmax(test_r2)], max(test_r2),
        marker='o', color = "red", label="max R^2")
plt.xticks(hiper_param)
plt.xlabel('Complejidad (n_estimator)')
plt.ylabel('R2')
plt.legend(loc = 'lower right')
plt.show()



In [ ]:
# Consideramos un rango para asignar el hiperparámetro
hiper_param = np.arange(2,30)

# Generamos previamente los vectores necesarios para ir calculando y guardando el rendimiento
test_r2 = np.zeros(hiper_param.size)

for i in range(hiper_param.size):
    # Generamos un modelo para cada hiperparámetro
    mod_bosque = ForecasterAutoreg(
                    regressor = RandomForestRegressor(max_depth=hiper_param[i],
                                                      n_estimators=10,
                                                      random_state=77),
                lags = [1,2,9,12,13]
                )

    mod_bosque.fit(y=df_train['passengers'])
    test_r2[i] = r2_score(df_test, mod_bosque.predict(steps=horizonte).round())

print("El mejor valor de max_depth podría ser =", hiper_param[np.argmax(test_r2)],
      " que consigue un R2 =", max(test_r2))

# Graficamos el R_cuadrado de training versus de test
fig = plt.figure(figsize=(20,7))
plt.plot(hiper_param, test_r2, linewidth=3, label='Test R^2')
plt.plot(hiper_param[np.argmax(test_r2)], max(test_r2),
        marker='o', color = "red", label="max R^2")
plt.xticks(hiper_param)
plt.xlabel('Complejidad (max_depth)')
plt.ylabel('R2')
plt.legend(loc = 'lower right')
plt.show()



In [ ]:
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg


mod_rf_opt = ForecasterAutoreg(
                regressor = RandomForestRegressor(n_estimators=10,
                                                  max_depth=10,
                                                  random_state=77),
                lags = [1,2,9,12,13]
                )

mod_rf_opt.fit(y=df_train['passengers'])
pred_rf_opt = mod_rf_opt.predict(steps=horizonte).round()

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_rf_opt))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_rf_opt)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_rf_opt))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_rf_opt)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_rf_opt))


In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_rf_opt, label='Predicción - RF (óptimo)')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

Gracias a la conversión realizada por _skforecast_ podemos aplicar distintas técnicas de ML que se pueden aplicar en los problemas de regresión.

### XGBoost

Esta vez usamos los modelos de _XGBoost_ para modelar la serie temporal.

In [ ]:
from xgboost import XGBRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg


mod_xgb = ForecasterAutoreg(
                regressor = XGBRegressor(random_state=77),
                lags = 12
                )

mod_xgb.fit(y=df_train['passengers'])
pred_xgb = mod_xgb.predict(steps=horizonte).round()

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_xgb))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_xgb)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_xgb))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_xgb)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_xgb))


In [ ]:
from skforecast.model_selection import grid_search_forecaster

forecaster = ForecasterAutoreg(
                regressor = XGBRegressor(random_state=77),
                lags      = 12
             )

# Lags
lags_grid = [12, 13, [1,2,9,12,13]]

# Parámetros del regresor
param_grid = {'n_estimators': [100, 300, 500],
              'max_depth': [5, 10, 20, 30],
              'learning_rate': [0.01, 0.1, 0.5, 1]}

results_grid_xgb = grid_search_forecaster(
                        forecaster         = forecaster,
                        y                  = df_train['passengers'],
                        param_grid         = param_grid,
                        lags_grid          = lags_grid,
                        steps              = horizonte,
                        refit              = False,
                        metric             = 'mean_absolute_error',
                        initial_train_size = int(len(df_train)*0.5),
                        fixed_train_size   = False,
                        return_best        = True,
                        verbose            = False
               )

In [ ]:
from xgboost import XGBRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg


mod_xgb_grid = ForecasterAutoreg(
                regressor = XGBRegressor(n_estimators=300,
                                         max_depth=10,
                                         learning_rate=0.1,
                                         random_state=77),
                lags = [1,2,9,12,13]
                )

mod_xgb_grid.fit(y=df_train['passengers'])
pred_xgb_grid = mod_xgb_grid.predict(steps=horizonte).round()

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_xgb_grid))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_xgb_grid)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_xgb_grid))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_xgb_grid)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_xgb_grid))


Ahora volvemos a analizar los hiperparámetros de uno en uno.

In [ ]:
# Consideramos un rango para asignar el hiperparámetro
hiper_param = np.arange(5,501,5)

# Generamos previamente los vectores necesarios para ir calculando y guardando el rendimiento
test_r2 = np.zeros(hiper_param.size)

for i in range(hiper_param.size):
    # Generamos un modelo para cada hiperparámetro
    mod_bosque = ForecasterAutoreg(
                    regressor = XGBRegressor(n_estimators=hiper_param[i],
                                                      random_state=77),
                lags = [1,2,9,12,13]
                )

    mod_bosque.fit(y=df_train['passengers'])
    test_r2[i] = r2_score(df_test, mod_bosque.predict(steps=horizonte).round())

print("El mejor valor de n_estimator podría ser =", hiper_param[np.argmax(test_r2)],
      " que consigue un R2 =", max(test_r2))

# Graficamos el R_cuadrado de training versus de test
fig = plt.figure(figsize=(20,7))
plt.plot(hiper_param, test_r2, linewidth=3, label='Test R^2')
plt.plot(hiper_param[np.argmax(test_r2)], max(test_r2),
        marker='o', color = "red", label="max R^2")
plt.xticks(hiper_param)
plt.xlabel('Complejidad (n_estimator)')
plt.ylabel('R2')
plt.legend(loc = 'lower right')
plt.show()



In [ ]:
# Consideramos un rango para asignar el hiperparámetro
hiper_param = np.arange(2,30)

# Generamos previamente los vectores necesarios para ir calculando y guardando el rendimiento
test_r2 = np.zeros(hiper_param.size)

for i in range(hiper_param.size):
    # Generamos un modelo para cada hiperparámetro
    mod_bosque = ForecasterAutoreg(
                    regressor = XGBRegressor(max_depth=hiper_param[i],
                                                      n_estimators=50,
                                                      random_state=77),
                lags = [1,2,9,12,13]
                )

    mod_bosque.fit(y=df_train['passengers'])
    test_r2[i] = r2_score(df_test, mod_bosque.predict(steps=horizonte).round())

print("El mejor valor de max_depth podría ser =", hiper_param[np.argmax(test_r2)],
      " que consigue un R2 =", max(test_r2))

# Graficamos el R_cuadrado
fig = plt.figure(figsize=(20,7))
plt.plot(hiper_param, test_r2, linewidth=3, label='Test R^2')
plt.plot(hiper_param[np.argmax(test_r2)], max(test_r2),
        marker='o', color = "red", label="max R^2")
plt.xticks(hiper_param)
plt.xlabel('Complejidad (max_depth)')
plt.ylabel('R2')
plt.legend(loc = 'lower right')
plt.show()



In [ ]:
from xgboost import XGBRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg


mod_xgb_grid = ForecasterAutoreg(
                regressor = XGBRegressor(n_estimators=50,
                                         max_depth=10,
                                         random_state=77),
                lags = [1,2,9,12,13]
                )

mod_xgb_grid.fit(y=df_train['passengers'])
pred_xgb_grid = mod_xgb_grid.predict(steps=horizonte).round()

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_xgb_grid))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_xgb_grid)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_xgb_grid))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_xgb_grid)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_xgb_grid))


In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_xgb_grid, label='Predicción - XGBRegressor (opt)')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

### LightGBM

LightGBM (`Light Gradient Boosting Machine`) es una herramienta de alto rendimiento desarrollado por _Microsoft_ que utiliza algoritmos de árboles de decisión para tareas de aprendizaje supervisado como regresión y clasificación.  

_LightGBM_ está escrito en __C++__, pero también tiene una interfaz de _Python_ y se puede usar junto con librerías populares de análisis de datos y aprendizaje automático, como pandas, _scikit-learn_ y _XGBoost_.

In [ ]:
from lightgbm import LGBMRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg

mod_lgbm = ForecasterAutoreg(
                regressor = LGBMRegressor(random_state=77),
                lags = 12
                )

mod_lgbm.fit(y=df_train['passengers'])
pred_lgbm = mod_lgbm.predict(steps=horizonte).round()

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_lgbm))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_lgbm)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_lgbm))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_lgbm)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_lgbm))


In [ ]:
from skforecast.model_selection import grid_search_forecaster

forecaster = ForecasterAutoreg(
                regressor = LGBMRegressor(random_state=77),
                lags      = 12
             )

# Lags used as predictors
lags_grid = [12, 13, [1,2,9,12,13]]

# Regressor's hyperparameters
param_grid = {'n_estimators': [100, 300, 500],
              'max_depth': [5, 10, 20, 30],
              'learning_rate': [0.01, 0.1, 0.5, 1]}

results_grid_lgbm = grid_search_forecaster(
                        forecaster         = forecaster,
                        y                  = df_train['passengers'],
                        param_grid         = param_grid,
                        lags_grid          = lags_grid,
                        steps              = horizonte,
                        refit              = False,
                        metric             = 'mean_absolute_error',
                        initial_train_size = int(len(df_train)*0.5),
                        fixed_train_size   = False,
                        return_best        = True,
                        verbose            = False
               )

In [ ]:
from skforecast.ForecasterAutoreg import ForecasterAutoreg

mod_lgbm_grid = ForecasterAutoreg(
                regressor = LGBMRegressor(n_estimators=100,
                                         max_depth=30,
                                         learning_rate=1,
                                         random_state=77),
                lags = [1,2,9,12,13]
                )

mod_lgbm_grid.fit(y=df_train['passengers'])
pred_lgbm_grid = mod_lgbm_grid.predict(steps=horizonte).round()

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_lgbm_grid))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_lgbm_grid)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_lgbm_grid))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_lgbm_grid)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_lgbm_grid))


In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_lgbm_grid, label='Predicción - LGBMRegressor (grid)')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

### CatBoost


_CatBoost_ es un framework open-source de _gradient boosting_ que está diseñado para funcionar bien especialmente con características categóricas sin necesidad de codificación one-hot u otros pasos de preprocesamiento.

In [ ]:
pip install catboost

In [ ]:
from catboost import CatBoostRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg


mod_cat = ForecasterAutoreg(
                regressor = CatBoostRegressor(random_state=77, silent=True),
                lags = 12
                )

mod_cat.fit(y=df_train['passengers'])
pred_cat = mod_cat.predict(steps=horizonte).round()

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_cat))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_cat)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_cat))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_cat)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_cat))


In [ ]:
from skforecast.model_selection import grid_search_forecaster

# Hyperparameter Grid search
# ==============================================================================
forecaster = ForecasterAutoreg(
                regressor = CatBoostRegressor(random_state=77, silent=True),
                lags      = 12
             )

# Lags used as predictors
lags_grid = [12, 13, [1,2,9,12,13]]

# Regressor's hyperparameters
param_grid = {'n_estimators': [100, 300, 500],
              'max_depth': [5, 10],
              'learning_rate': [0.01, 0.1, 1]}

results_grid_cat = grid_search_forecaster(
                        forecaster         = forecaster,
                        y                  = df_train['passengers'],
                        param_grid         = param_grid,
                        lags_grid          = lags_grid,
                        steps              = horizonte,
                        refit              = False,
                        metric             = 'mean_absolute_error',
                        initial_train_size = int(len(df_train)*0.5),
                        fixed_train_size   = False,
                        return_best        = True,
                        verbose            = False
               )

In [ ]:
from skforecast.ForecasterAutoreg import ForecasterAutoreg

mod_cat_grid = ForecasterAutoreg(
                regressor = CatBoostRegressor(n_estimators=300,
                                              max_depth=5,
                                              learning_rate=0.1,
                                              random_state=77,
                                              silent=True),
                lags = [1,2,9,12,13]
                )

mod_cat_grid.fit(y=df_train['passengers'])
pred_cat_grid = mod_cat_grid.predict(steps=horizonte).round()

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_cat_grid))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_cat_grid)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_cat_grid))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_cat_grid)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_cat_grid))


In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_cat_grid, label='Predicción - CatBoostRegressor (grid)')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

### Bayesian search

La búsqueda en _grid_ puede generar buenos resultados, especialmente cuando se reduce el rango de búsqueda. Sin embargo, **no se tienen en cuanta los resultados obtenidos** en cada experimento, lo que **impide centrar la búsqueda en las regiones de mayor interés** para evitar los cálculos innecesarios.

Una alternativa es utilizar **métodos de optimización bayesianos** para buscar hiperparámetros. En términos generales, la optimización de hiperparámetros bayesianos consiste en crear un modelo probabilístico en el que la función objetivo es la métrica de validación del modelo (RMSE, AUC, precisión...). Con esta estrategia, **la búsqueda se redirige en cada iteración a las regiones de mayor interés**. El objetivo final es **reducir el número de combinaciones de hiperparámetros** con las que se evalúa el modelo, eligiendo solo los mejores candidatos.

_Skforecast_ ofrece algunos motores de optimización bayesianos. Aquí usamos un ejempo el método **`Optuna`**. La búsqueda bayesiana solo se aplica a los hiperparámetros del modelo, y no a los retrasos, ya que se evalúan todos los _lags_ especificados por el usuario.

In [ ]:
from skforecast.model_selection import  bayesian_search_forecaster

forecaster = ForecasterAutoreg(
                 regressor = XGBRegressor(random_state=77),
                 lags      = 12
             )

# Lags
lags_grid = [12, 13, [1,2,9,12,13]]

# Parámetros del regresor
def search_space(trial):
    search_space  = {'n_estimators'     : trial.suggest_int('n_estimators', 100, 500, 100),
                     'max_depth'        : trial.suggest_int('max_depth', 5, 30, 5)}
    return search_space

results, frozen_trial = bayesian_search_forecaster(
                            forecaster            = forecaster,
                            y                     = df_train['passengers'],
                            lags_grid             = lags_grid,
                            search_space          = search_space,
                            steps                 = horizonte,
                            metric                = 'mean_absolute_error',
                            refit                 = True,
                            initial_train_size    = int(len(df_train)*0.5),
                            fixed_train_size      = True,
                            n_trials              = 10,
                            random_state          = 77,
                            return_best           = True,
                            verbose               = False,
                            engine                = 'optuna',
                            kwargs_create_study   = {},
                            kwargs_study_optimize = {}
                        )

In [ ]:
from xgboost import XGBRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg

mod_xgb_grid_opt = ForecasterAutoreg(
                regressor = XGBRegressor(n_estimators=400,
                                         max_depth=5,
                                         random_state=77),
                lags = [1,2,9,12,13]
                )

mod_xgb_grid_opt.fit(y=df_train['passengers'])
pred_xgb_grid_opt = mod_xgb_grid_opt.predict(steps=horizonte).round()

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_test, pred_xgb_grid_opt))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_test, pred_xgb_grid_opt)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_test, pred_xgb_grid_opt))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_test, pred_xgb_grid_opt)))
print('R^2 coefficient of determination:', r2_score(df_test, pred_xgb_grid_opt))


In [ ]:
plt.plot(df_test, label='Test')
plt.plot(pred_xgb_grid_opt, label='Predicción - XGBRegressor (grid con Optuna)')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

---

### **`Ejercicio 26`**

Vamos a analizar los datos de **`Sunspots Dataset`** que son números promediados mensuales de **manchas solares desde 1749 hasta 1983**.  


- Utilizamos el siguiente enlace para descargar los datos y crear una tabla tipo _DataFrame_ con ellos:
  - 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'  
  

- Construimos la serie temporal, del modo que las fechas tipo `'1749-05-01'` formen los índices, y número de las manchas solares los valores de la serie.


---

In [ ]:
# Solución 25.1.1

import pandas as pd
import matplotlib.pyplot as plt

# Load the Air Quality dataset
df_spot = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv', index_col=False)
df_spot


In [ ]:
# Solución 25.1.2
# Convertir la columna en fechas tipo DatetimeIndex
df_spot['fechas'] = pd.to_datetime(df_spot['Month'], format='%Y-%m')

# Convertir la columna de fechas a los índices del DataFrame
df_spot.set_index('fechas', inplace=True)
df_spot.index.freq = 'MS'

# Remove the "year" and "month" columns
df_spot.drop(columns='Month', inplace=True)

df_spot

In [ ]:
# Visualizar el DataFrame creado con los datos de la serie temporal
plt.plot(df_spot)
plt.show()

In [ ]:
# Definir el periodo de prueba (horizonte de predicción)
horizonte = 12  # La cantidad de puntos a predecir
df_spot_test = df_spot.tail(horizonte)
df_spot_test

In [ ]:
# Filtrar la serie original para sacar el periodo de entrenamiento
df_spot_train = df_spot[df_spot.index.isin(df_spot_test.index)==False]
df_spot_train

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

rcParams['figure.figsize'] = 14, 7
plot_acf(df_spot, lags=50)
plt.xticks(np.arange(50))
plt.ylim(-1.1,1.1)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

plot_pacf(df_spot, lags=50, method='ywm')
plt.xticks(np.arange(50))
plt.ylim(-1.1,1.1)
plt.show()

Vemos que los retrasos con algo de autocorrelación parcial pueden llegar hasta el lag 34 aproximadamente.

In [ ]:
# Consideramos un rango para asignar el hiperparámetro
hiper_param = np.arange(10,301,50)

# Generamos previamente los vectores necesarios para ir calculando y guardando el rendimiento
test_r2 = np.zeros(hiper_param.size)

for i in range(hiper_param.size):
    # Generamos un modelo para cada hiperparámetro
    mod_bosque = ForecasterAutoreg(
                    regressor = RandomForestRegressor(n_estimators=hiper_param[i],
                                                      random_state=77),
                lags = 34
                )

    mod_bosque.fit(y=df_spot_train['Sunspots'])
    test_r2[i] = r2_score(df_spot_test, mod_bosque.predict(steps=horizonte).round())

print("El mejor valor de lags podría ser =", hiper_param[np.argmax(test_r2)],
      " que consigue un R2 =", max(test_r2))

# Graficamos el R_cuadrado
fig = plt.figure(figsize=(20,7))
plt.plot(hiper_param, test_r2, linewidth=3, label='Test R^2')
plt.plot(hiper_param[np.argmax(test_r2)], max(test_r2),
        marker='o', color = "red", label="max R^2")
plt.xticks(hiper_param)
plt.xlabel('Complejidad (lags)')
plt.ylabel('R2')
plt.legend(loc = 'lower right')
plt.show()



In [ ]:
# Consideramos un rango para asignar el hiperparámetro
hiper_param = np.arange(2,30)

# Generamos previamente los vectores necesarios para ir calculando y guardando el rendimiento
test_r2 = np.zeros(hiper_param.size)

for i in range(hiper_param.size):
    # Generamos un modelo para cada hiperparámetro
    mod_bosque = ForecasterAutoreg(
                    regressor = RandomForestRegressor(max_depth=hiper_param[i],
                                                      n_estimators=110,
                                                      random_state=77),
                lags = 34
                )

    mod_bosque.fit(y=df_spot_train['Sunspots'])
    test_r2[i] = r2_score(df_spot_test, mod_bosque.predict(steps=horizonte).round())

print("El mejor valor de max_depth podría ser =", hiper_param[np.argmax(test_r2)],
      " que consigue un R2 =", max(test_r2))

# Graficamos el R_cuadrado
fig = plt.figure(figsize=(20,7))
plt.plot(hiper_param, test_r2, linewidth=3, label='Test R^2')
plt.plot(hiper_param[np.argmax(test_r2)], max(test_r2),
        marker='o', color = "red", label="max R^2")
plt.xticks(hiper_param)
plt.xlabel('Complejidad (max_depth)')
plt.ylabel('R2')
plt.legend(loc = 'lower right')
plt.show()



In [ ]:
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg


mod_spot_rf = ForecasterAutoreg(
                regressor = RandomForestRegressor(n_estimators=110,
                                                  max_depth=20,
                                                  random_state=77),
                lags = 34
                )

mod_spot_rf.fit(y=df_spot_train['Sunspots'])
pred_spot_rf = mod_spot_rf.predict(steps=horizonte).round()

# Métricas de evaluación del modelo
print('Mean Absolute Error (MAE):', mean_absolute_error(df_spot_test, pred_spot_rf))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_spot_test, pred_spot_rf)*100)
print('Mean Squared Error (MSE):', mean_squared_error(df_spot_test, pred_spot_rf))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_spot_test, pred_spot_rf)))
print('R^2 coefficient of determination:', r2_score(df_spot_test, pred_spot_rf))


In [ ]:
plt.plot(df_spot_test, label='Test')
plt.plot(pred_spot_rf, label='Predicción - RF (opt)')
plt.title("Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)")
plt.legend()
plt.show()

---

### **`Ejercicio 26.1`**

Vamos a intentar a modelar la serie temporal con el objetivo de predecir los valores mensuales del último año.

**`26.1.1`** Realiza una búsqueda mediante _Compexity Curve_ para encontrar el número óptimo de retrasos, teniendo en cuenta los siguientes puntos:

- Hasta `lag_34` inclusive
- Modelo: **XGBoost**
- `random_state=77`

**`26.1.2`** Teniendo en cuenta el valor óptimo calculado en el paso anterior para los retrasos, realiza una búsqueda mediante _Compexity Curve_ para encontrar el número óptimo de **árboles**.

**`26.1.3`** Teniendo en cuenta los valores óptimos calculados en los pasos anteriores, realiza una búsqueda mediante _Compexity Curve_ para encontrar el número óptimo de la **profundidad máxima**.

**`26.1.4`** Construye un modelo con los hiperparámetros óptimos que hayas calculado y calcula las métricas de calidad del modelo y de tus predicciones:

- Las métricas de "_Out-of-sample performance_": MAE, MAPE, MSE, RMSE y R2.

**`26.1.5`** Saca la gráfica de "_Datos reales vs. Predicción del modelo (Out-of-Sample forecasting)_" y **analiza y compara los resultados** de este modelo con el último modelo contruido en la sesión.

---